# Chainerで線形回帰してみる

線形回帰についてはtensorflowを参考。ここではChainer独自のところを説明していく

In [1]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, Variable, optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L

ChainerでもVariableを使う

In [2]:
x_data = np.array([2], dtype=np.float32)
x = Variable(x_data)
print(x)
print("x.data ->", x.data)
print("x.grad ->", x.grad) # この段階でgradはNone
y = x ** 2
print("y.data->", y.data)

<var@11137b6d8>
x.data -> [ 2.]
x.grad -> None
y.data-> [ 4.]


アウトプットに対して、`backward()`とすると、その関数を作っているVariableでのgradの値が入力される。この場合はxでの偏微分の値。

In [3]:
y.backward()
print(x.grad)
y.zerograd() # 偏微分の値は蓄積されていくので、zerogradで初期化

[ 4.]


Chainerが面白いのは、F.LinerやL.Linerが
$$ \vec{y} = W\vec{x} +\vec{b}$$
の形を持った関数で、$W$や$\vec{b}$はランダムに初期化してくれている。

`F.Liner(1,1)`だと、1次元→1次元で、あとでMNISTなどで使う784次元→10次元であれば`F.Liner(784,10)`とすれば、&W&は784x10の行列で、$\vec{b}\in\mathbb{R}^{10}$となるように作ってくれる

In [4]:
f = F.Linear(1,1) # 1次元→1次元の y = Wx + b
print(f.W.data)
print(f.b.data)

[[ 1.33073533]]
[ 0.]


100個のデータセットを作って、コスト関数を平均二乗誤差関数で定義してみる。

In [5]:
x_data = np.random.rand(100,1).astype("float32")
y_data = x_data * 0.1 + 0.3
x = Variable(x_data)
y = Variable(y_data)
loss = F.mean_squared_error(f(x), y) # 平均二乗誤差
loss.data

array(0.27102425694465637, dtype=float32)

バッチで一気に計算すると器用に偏微分の値は蓄積されていくみたいなので、偏微分を計算する前に初期化しておくための`zerograds`が準備されているので呼ぶ。

In [6]:
f.zerograds() # 微分値の初期化
loss.backward() # それぞれの偏微分を計算する
print(f.W.grad) # fのWでの変微分の値が求まる
print(f.b.grad) # fのbでの変微分の値が求まる

[[ 0.62662262]]
[ 0.7638604]


コスト関数の最小化の手法はoptimizersに準備されている。最急降下法がなさそうなので、確率的最急降下法を使う。`setup`で変数の更新対象の関数をセットする。そして、`update`でパラメーターを更新してくれる。

In [7]:
optimizer = optimizers.SGD(0.5) # 確率的最急降下法
optimizer.setup(f)
optimizer.update()
print(f.W.data)
print(f.b.data)

[[ 1.01742399]]
[-0.3819302]


上記を合わせてイテレーションして学習を進めてみる

In [8]:
for step in range(201):
    loss = F.mean_squared_error(f(x), y)  # 再度コスト関数を計算する
    f.zerograds() # gradは保存されていくので初期化する
    loss.backward() # 微分を計算
    optimizer.update() # 確率的勾配降下法で fをupdateする
    if step % 20 == 0:
        print(step,f.W.data, f.b.data)

0 [[ 1.03781068]] [-0.20832953]
20 [[ 0.35380483]] [ 0.15003617]
40 [[ 0.17018947]] [ 0.25852767]
60 [[ 0.11941081]] [ 0.28853089]
80 [[ 0.10536806]] [ 0.29682824]
100 [[ 0.10148452]] [ 0.29912287]
120 [[ 0.10041054]] [ 0.29975745]
140 [[ 0.10011356]] [ 0.29993293]
160 [[ 0.10003141]] [ 0.29998147]
180 [[ 0.10000867]] [ 0.29999489]
200 [[ 0.10000242]] [ 0.29999858]


綺麗に、学習されている。

次に、Chainというクラスを使うことにより、もう少し便利に、上記が出来ることを見ていく。深層学習などの複雑なニューラルネットワークを生成するには、こちらを使うほうが便利そうだ。やっていることは上記と同じなので、説明は省く。

In [9]:
class LinearRegression(Chain):
    def __init__(self):
        super(LinearRegression, self).__init__(
            l1 = L.Linear(1,1)
        )
        
    def __call__(self, x, y):
        return F.mean_squared_error(self.l1(x), y)

In [10]:
model = LinearRegression()
optimizer = optimizers.SGD(0.5)
optimizer.setup(model)

In [11]:
import numpy as np
x_data = np.random.rand(100,1).astype("float32")
y_data = x_data * 0.1 + 0.3
print("x_data -> ", x_data[:5])
print("y_data -> ", y_data[:5])

x_data ->  [[ 0.36174315]
 [ 0.45916244]
 [ 0.55743873]
 [ 0.20309481]
 [ 0.90550619]]
y_data ->  [[ 0.33617434]
 [ 0.34591627]
 [ 0.35574389]
 [ 0.32030949]
 [ 0.39055064]]


In [12]:
for step in range(201):
    train_x = Variable(x_data)
    train_y = Variable(y_data)
    optimizer.update(model, train_x, train_y)
    if step % 20 == 0:
        print(step, model.l1.W.data, model.l1.b.data) 
    
#     model.zerograds()
#     loss = model(train_x, train_y)
#     loss.backword()
#     optimizer.update()

0 [[ 0.7579459]] [-0.05036949]
20 [[ 0.28956166]] [ 0.20411299]
40 [[ 0.15403411]] [ 0.27266765]
60 [[ 0.1154023]] [ 0.29220897]
80 [[ 0.10439041]] [ 0.2977792]
100 [[ 0.10125148]] [ 0.29936698]
120 [[ 0.10035673]] [ 0.29981956]
140 [[ 0.10010167]] [ 0.29994857]
160 [[ 0.10002899]] [ 0.29998535]
180 [[ 0.10000826]] [ 0.29999584]
200 [[ 0.10000234]] [ 0.29999882]


綺麗に学習できている。

# MNISTをパーセプトロンで学習してみる

In [13]:
import pickle, gzip
f = gzip.open('../data/mnist.pkl.gz', 'rb')
train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
print(len(train_set[0]), len(valid_set[0]), len(test_set[0]))
train_set_x, train_set_y  = train_set
test_set_x, test_set_y = test_set
print(len(train_set_x[0]))
print(train_set_x)
print("28x28の画像のモノクロ(白0→黒1)が１次元配列で入っている",train_set_x[:5])
print("最初のラベルは",train_set_y[0])

# あとで、softmax_cross_entropyを使うときに型の判定があり、np.int32じゃないといけない
train_set_y = train_set_y.astype(np.int32)
test_set_y = test_set_y.astype(np.int32)
print(train_set_y )
f.close()

50000 10000 10000
784
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
28x28の画像のモノクロ(白0→黒1)が１次元配列で入っている [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
最初のラベルは 5
[5 0 4 ..., 8 4 8]


In [14]:
class MNISTPerceptronClassifier(Chain):
    def __init__(self):
        super(MNISTPerceptronClassifier, self).__init__(
            l1 = L.Linear(784, 10)
        )
        
    def __call__(self, x, y):
        pridict = self.l1(x)
        self.loss = F.softmax_cross_entropy(pridict, y)
        self.accuracy = F.accuracy(pridict, y) 
        return self.loss
    

In [15]:
model = MNISTPerceptronClassifier()
optimizer = optimizers.SGD(0.1) # 確率的勾配降下法なので、ちょっと大きめに
optimizer.setup(model)

In [16]:
batch_size = 100
for step in range(1000):
    batch_index = np.random.randint(len(train_set[0])-batch_size)
    batch_x = Variable(train_set_x[batch_index:batch_index+batch_size])
    batch_y = Variable(train_set_y[batch_index:batch_index+batch_size])
    optimizer.update(model, batch_x, batch_y)

In [22]:
model(Variable(test_set_x),Variable(test_set_y))
print(model.accuracy.data)

0.907800018787384


90%ぐらいの制度がでる

公式のサンプルを見ると、Chainでは、ネットワークの構成だけにしておいて、L.Classifierを使うのが便利そう。[ドキュメント](http://docs.chainer.org/en/stable/_modules/chainer/links/model/classifier.html)を見ると、ロス関数としてソフトマックス関数が設定されていたり、`self.accuracy`も設定されているので、便利に使える。イニシャライズするときに、変数で渡せば任意のものが使える。分類をするときは最後にこれをつければ良さそう

In [23]:
class MNISTPerceptron(Chain):
    def __init__(self):
        super(MNISTPerceptron, self).__init__(
            l1 = L.Linear(784, 10)
        )
        
    def __call__(self, x):
        return self.l1(x)
    
model = L.Classifier(MNISTPerceptron())
optimizer.setup(model)
batch_size = 100
for step in range(1000):
    batch_index = np.random.randint(len(train_set[0])-batch_size)
    batch_x = Variable(train_set_x[batch_index:batch_index+batch_size])
    batch_y = Variable(train_set_y[batch_index:batch_index+batch_size])
    optimizer.update(model, batch_x, batch_y)
    
model(Variable(test_set_x),Variable(test_set_y))
print(model.accuracy.data)

0.9075000286102295
